# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import time
%matplotlib inline

In [1]:
from utils import StarbucksProject

In [2]:
SP = StarbucksProject()

[2023-11-26, 03:16:04] - [Data Import] Importation of Data - Done!


In [3]:
SP.fit_preprocess()

[2023-11-26, 03:16:19] - [Feature Engineering] Profile - Done!
[2023-11-26, 03:16:19] - [Feature Engineering] Transcript - Done!
[2023-11-26, 03:16:19] - [Feature Engineering] Portfolio - Done!


/home/ec2-user/SageMaker/StarbucksCapstoneProject/utils.py:144: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  channels = pd.get_dummies(self.portfolio.channels.apply(pd.Series).stack()).sum(level=0)


[2023-11-26, 03:16:20] - [Reverse Engineering] Offers + Transactions - Done!
[2023-11-26, 03:16:21] - [Build Query] Offers Completed - Done!
[2023-11-26, 03:16:21] - [Processing Views] Back-up Retrieved - Done!
[2023-11-26, 03:16:21] - [Processing Completed] Adding Amount Value - Done!
[2023-11-26, 03:16:22] - [Processing Views] Building Views Table - Done!
[2023-11-26, 03:16:22] - [Processing Receive] Building Receive Table - Done!
[2023-11-26, 03:16:22] - [Processing Transaction] Building Transactional Table - Done!


/home/ec2-user/SageMaker/StarbucksCapstoneProject/utils.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.transcript_received[['time_view', 'time_completion', 'time_for_completion', 'time_for_view']] = None


[2023-11-26, 03:16:26] - [Processing Dataset] Building Gathered Table - Done!


In [5]:
SP.transcript.head()

,customer_id,event,time,amount,offer_id,premium
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [7]:
SP.profile.head()

,gender,age,customer_id,income,year,month,day
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017,2,12
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017,7,15
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,NaN,2018,7,12
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017,5,9
4,None,118,a03223e636434f42ac4c3df47e8bac43,NaN,2017,8,4


In [6]:
SP.portfolio.head()

,reward,difficulty,duration,offer_type,offer_id,email,mobile,social,web
0,10,10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,96,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,168,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,240,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1


In [9]:
SP.complete_table.head()

,customer_id,offer_id,time_receive,time_view,time_completion,time_for_completion,time_for_view,general_journey,amount,gender,...,month,day,reward,difficulty,duration,offer_type,email,mobile,social,web
0,6242d849cb3e401aa398ef3aefd21846,None,None,None,48,None,None,transactioned,7.54,F,...,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22ada0ed66d34eb19a42463454b9e75c,None,None,None,204,None,None,transactioned,1.5,None,...,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e7593c299cd84b9f9b2c5d170ebc36df,None,None,None,318,None,None,transactioned,4.6,None,...,1,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,48dbc372c7164fe99b9a90f0249b6024,None,None,None,432,None,None,transactioned,1.5,M,...,2,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72cb17e8c761445f8b1a6782c39c9831,None,None,None,450,None,None,transactioned,3.72,M,...,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Profile EDA

In [18]:
# fig, axs = plt.subplots(2,2)

# sns.kdeplot(profile, hue = 'gender', x = 'age', ax = axs[0][0])
# sns.kdeplot(profile, hue = 'gender', x = 'income', ax = axs[0][1])
# sns.histplot(profile, x = 'gender', ax = axs[1][0])
# sns.histplot(profile, x = 'year', ax = axs[1][1])

# fig.set_figheight(10)
# fig.set_figwidth(14)

# plt.savefig('images/profile_eda.png')

In [19]:
# Offers EDA

In [20]:
# df_offers = transcript.dropna(subset=['offer_id']).drop(['reward'], axis = 1).merge(portfolio.drop(['channels'], axis = 1), left_on = 'offer_id', right_on = 'id', how = 'inner')

# fig, axs = plt.subplots(2,2)

# sns.histplot(df_offers, x = 'reward', ax = axs[0][0])
# sns.histplot(df_offers, x = 'difficulty', ax = axs[0][1])
# sns.histplot(df_offers, x = 'duration', ax = axs[1][0])
# sns.histplot(df_offers, x = 'offer_type', ax = axs[1][1])
# fig.set_figheight(10)
# fig.set_figwidth(14)

# plt.savefig('images/offers_eda.png')